In [1]:
import pandas as pd
import ngrams
!pip install vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import numpy as np
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('words')
from nltk import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import re
from nltk.corpus import words

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [13]:
l=20 #set length
df = pd.read_csv('./datasetV3.csv')[:l]
columns = df.columns
df.head(5)

,Data,Date,Time,tweet0,tweet1,tweet2,tweet3,tweet4,tweet5,tweet6,tweet7,tweet8,tweet9
0,tuesdayvibes,7/14/2020,7:00:21,Love it here vacation vibes amazing beautiful ...,That looks so good tuesdayvibes tacotuesday,LIVE NOW TUESDAY NIGHT KPOP W Unreal NOW OHIO ...,My damn followers haven t moved in weeks I m g...,Common sense is not that common You can t coun...,Just landing naturephotography nature Travel t...,LIVE NOW TUESDAY NIGHT KPOP W Unreal NOW SUMME...,Involuntary manslaughter is distinguished from...,How was your day keepgoing throughitall carryo...,Happy scrolls down on the notification bar to ...
1,realmeC11,7/14/2020,7:00:21,Best Camera Smartphone under 20k Please vote a...,2 features in highlight realmeC11 in the realm...,realmeC11 Bigger battery and larger display,1 It is backed by a bigger 5000 mAh battery 2 ...,1 Bigger Battery 5000mAh Massive Battery 2 Lar...,4th phone under Realme C series RealmeC11,Hey its high time to release new watch faces f...,The new entry of the realme C series boasts a ...,realmeC11 Price is too much bro realmeC11 ki M...,The new entry to the realme C series boasts a ...
2,KPSharmaOli,7/14/2020,7:00:21,shree Why should we have a problem with the pe...,It s really insulting that Nepal PM has said w...,We always loved amp respect India but see what...,This is it Ayodhya KPSharmaOli,Watch Gravitas Nepal KP Sharma Oli s desperate...,is minting money and making foolish to innocen...,diya That means all well doing nepalese immigr...,st No make sense how about Pakistani Maybe you...,Intelligent PM Bought 6 Chinese planes that Dh...,st India offered Nepal to access port with fre...
3,RheaChakraborty,7/14/2020,7:00:21,Rhea Chakraborty s Heartbreaking Post On Susha...,Trending Entertainment News Today Rhea Chakrab...,Trending Entertainment News Today Rhea Chakrab...,Rhea Chakraborty breaks her silence on Sushant...,Support your bravery we are with you amp we wo...,Rhea Chakraborty pens an emotional note on los...,Newswrap July 14 Rhea s note for Sushant Singh...,Rhea Chakraborty s heartfelt post for Sushant ...,SushantSinghRajput No amount of PR of Rhea Cha...,Why r u silent if u really loves him then why ...
4,Stop_Transfer_Sunita_Yadav,7/14/2020,7:00:21,We stand for Sunita Yadav Stop the Transfer Wh...,CM of Gujrat set an example punish MLA not the...,I proud of you madam Stop Transfer Sunita Yadav,Thank u so much for ur valuable information Od...,I support Stop Transfer Sunita Yadav SunitaYad...,Stop Transfer Sunita Yadav We stand with U i s...,yadav 0 SunitaYadav stop transfer,hope it was passion to do than slogging for a ...,Stop Transfer Sunita Yadav if you tell truth u...,Immediate transfer of an employee after such e...


In [14]:
def separate_words(s):                         #to separate joined words
  return (ngrams.segment2(s)[1])
def sento(text):                               #to get the sentiment of text as a list -> [neg, neu, pos, compound]
  return np.array(list(SentimentIntensityAnalyzer().polarity_scores(text).values()))

In [15]:
en_stopwords = stopwords.words('english')
html_pattern = re.compile('<.*?>')
url_pattern = re.compile(r'https?://\S+|www\.\S+')
def preprocess(s):
  s= html_pattern.sub(r'', s)         #remove html tags(if present)
  s= url_pattern.sub(r'', s)          #remove url if present
  tokenizer = RegexpTokenizer(r"\w+")
  s=" ".join(str.lower(s).split())    #remove whitespace and transform to lower case
  s=(tokenizer.tokenize(s))           #remove punctuation and tokenize
  tmp=[]
  i=0
  dele=[]
  for i in s:
    tmp = tmp + separate_words(i)
  s = tmp
  for i in s:
    if(i not in SentimentIntensityAnalyzer().lexicon_full_filepath):      #delete all those words that vader doesnot recognize
      dele.append(i)
  for i in dele:
    s.remove(i)
  return ' '.join(s)

In [16]:
sentiments=[]
for i in df.columns[3:]:
    sentiments.append(np.array(df[i].apply(preprocess).apply(sento)))

In [17]:
sentiments = np.array(sentiments)

In [18]:
tmp=[]
for i in np.sum(sentiments, axis = 0):
    tmp.append(list(i))

In [19]:
sntmnt = pd.DataFrame(tmp, columns = ['neg', 'neu', 'pos', 'compound'])
df[:len(sntmnt)].join(sntmnt)

,Data,Date,Time,tweet0,tweet1,tweet2,tweet3,tweet4,tweet5,tweet6,tweet7,tweet8,tweet9,neg,neu,pos,compound
0,tuesdayvibes,7/14/2020,7:00:21,Love it here vacation vibes amazing beautiful ...,That looks so good tuesdayvibes tacotuesday,LIVE NOW TUESDAY NIGHT KPOP W Unreal NOW OHIO ...,My damn followers haven t moved in weeks I m g...,Common sense is not that common You can t coun...,Just landing naturephotography nature Travel t...,LIVE NOW TUESDAY NIGHT KPOP W Unreal NOW SUMME...,Involuntary manslaughter is distinguished from...,How was your day keepgoing throughitall carryo...,Happy scrolls down on the notification bar to ...,0.542,6.668,2.790,1.9998
1,realmeC11,7/14/2020,7:00:21,Best Camera Smartphone under 20k Please vote a...,2 features in highlight realmeC11 in the realm...,realmeC11 Bigger battery and larger display,1 It is backed by a bigger 5000 mAh battery 2 ...,1 Bigger Battery 5000mAh Massive Battery 2 Lar...,4th phone under Realme C series RealmeC11,Hey its high time to release new watch faces f...,The new entry of the realme C series boasts a ...,realmeC11 Price is too much bro realmeC11 ki M...,The new entry to the realme C series boasts a ...,0.284,8.842,0.874,1.5564
2,KPSharmaOli,7/14/2020,7:00:21,shree Why should we have a problem with the pe...,It s really insulting that Nepal PM has said w...,We always loved amp respect India but see what...,This is it Ayodhya KPSharmaOli,Watch Gravitas Nepal KP Sharma Oli s desperate...,is minting money and making foolish to innocen...,diya That means all well doing nepalese immigr...,st No make sense how about Pakistani Maybe you...,Intelligent PM Bought 6 Chinese planes that Dh...,st India offered Nepal to access port with fre...,1.364,7.035,1.601,-0.1506
3,RheaChakraborty,7/14/2020,7:00:21,Rhea Chakraborty s Heartbreaking Post On Susha...,Trending Entertainment News Today Rhea Chakrab...,Trending Entertainment News Today Rhea Chakrab...,Rhea Chakraborty breaks her silence on Sushant...,Support your bravery we are with you amp we wo...,Rhea Chakraborty pens an emotional note on los...,Newswrap July 14 Rhea s note for Sushant Singh...,Rhea Chakraborty s heartfelt post for Sushant ...,SushantSinghRajput No amount of PR of Rhea Cha...,Why r u silent if u really loves him then why ...,1.581,5.944,2.477,1.5946
4,Stop_Transfer_Sunita_Yadav,7/14/2020,7:00:21,We stand for Sunita Yadav Stop the Transfer Wh...,CM of Gujrat set an example punish MLA not the...,I proud of you madam Stop Transfer Sunita Yadav,Thank u so much for ur valuable information Od...,I support Stop Transfer Sunita Yadav SunitaYad...,Stop Transfer Sunita Yadav We stand with U i s...,yadav 0 SunitaYadav stop transfer,hope it was passion to do than slogging for a ...,Stop Transfer Sunita Yadav if you tell truth u...,Immediate transfer of an employee after such e...,2.774,4.716,2.509,1.2670
5,ArrestBishopFranco,7/14/2020,7:00:21,ArrestBishopFranco He should be arrested and t...,ArrestBishopFranco Now Without delay,Agreed ArrestBishopFranco,Justice is must ArrestBishopFranco,ArrestBishopFranco The nuns rapist twitter com...,ArrestBishopFranco Wonder where are the missio...,A man acqused of raping Nuns amp allegedly kee...,9879 ArrestBishopFranco Justice to nuns,Uphold the trust of people in Indian instituti...,Justice must prevail ArrestBishopFranco Arrest...,4.245,3.418,2.336,-2.3566
6,CWC19Final,7/14/2020,7:00:21,My heart skipped a beat when I watched the hig...,ThisDayLastYear The CWC19Final was over I was ...,A deflection like none other and in terms of c...,Moments after the ecstasy CWC19Final,One year ago today I experienced the biggest h...,onthisday 14 July 2019 England won the Cricket...,It was around this time 12 months ago that I l...,Yes yes I did where my England CWC19Final shir...,Today we recorded the latest FTP podcast with ...,Truly one of the most exhilarating days in cri...,0.258,6.662,3.079,4.8132
7,armyisoverparty,7/14/2020,7:00:21,nchild Chill Antis try to make armyisoverparty...,Here to help t